In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

In [ ]:
from config import config
import pandas
import seaborn
import numpy as np
import scipy
import matplotlib.pyplot as plt
import papermill as pm
from stacked_data import stacked_data, stacked_single
from tqdm import tqdm
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, IsolationForest, RandomForestClassifier, VotingClassifier
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from evaluate import calculate_confusion_matrix_stats, plot_roc_curve, plot_precision_recall, plot_confusion_matrix_ensemble

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Ensemble models

In [ ]:
# Parameters
MODELS = ['eff7e2af-4da0-41c4-b3a8-8e7140093544', 'e9d327c6-64f2-48e5-a666-fa0ce7eb9dc6', 'f5809fd4-ba72-4ea6-929a-44a42ba8c6fe']
SCORE = "accuracy"


In [ ]:
models = MODELS
print("models: {}".format(models))

In [ ]:
test_1_set, test_1_labels, test_1_dataset = stacked_single(
    models, 
    pickle="/data/intrepidlemon/renal/features/segmenter-3/training-features.pkl", 
    source="/data/intrepidlemon/renal/extra-segmentations/segmenter-3/"
)
test_2_set, test_2_labels, test_2_dataset = stacked_single(
    models, 
    pickle="/data/intrepidlemon/renal/features/segmenter-4/training-features.pkl", 
    source="/data/intrepidlemon/renal/extra-segmentations/segmenter-4/"
)
test_1_set = np.array(list(zip(*test_1_set)))
test_1_labels = np.array(test_1_labels)
test_2_set = np.array(list(zip(*test_2_set)))
test_2_labels = np.array(test_2_labels)

In [ ]:
sd = stacked_data(uuids=models)

In [ ]:
train_set = np.array(list(zip(*sd[0])))
validation_set = np.array(list(zip(*sd[1])))
test_set = np.array(list(zip(*sd[2])))
train_labels = np.array(sd[3])
validation_labels = np.array(sd[4])
test_labels = np.array(sd[5])
train_fix_set = np.array(list(zip(*sd[6])))
train_fix_labels = np.array(sd[7])

In [ ]:
from scipy.stats import uniform
CLASSIFIERS = [
    ExtraTreesClassifier,
#    AdaBoostClassifier, 
#    BaggingClassifier, 
#    GradientBoostingClassifier, 
#    RandomForestClassifier, 
]
PARAMETERS = [
    dict(bootstrap=[False], criterion=['gini', 'entropy'],
                     max_depth=[None, 2, 3, 4, 5], max_features=['auto'], max_leaf_nodes=[None, 5, 10],
                     min_impurity_decrease=uniform(0, 1), min_impurity_split=[None],
                     min_samples_leaf=[1], min_samples_split=[2],
                     min_weight_fraction_leaf=uniform(0, 0.5), n_estimators=[10, 50, 100, 500, 1000], n_jobs=[5],
                     oob_score=[False], random_state=[316], verbose=[0],
                     warm_start=[False]), 
    dict(),   
    dict(n_estimators=20, max_samples=0.1, max_features=1),
    dict(),     
    dict(),     
]
CLASSIFIERS = [
    ExtraTreesClassifier,
    AdaBoostClassifier, 
    BaggingClassifier, 
    GradientBoostingClassifier, 
    RandomForestClassifier, 
]
PARAMETERS = [
    dict(max_leaf_nodes=5, max_depth=2), 
    dict(),   
    dict(n_estimators=20, max_samples=0.1, max_features=1),
    dict(),     
    dict(),     
]

In [ ]:
def score_accuracy(model, x, y): 
    return metrics.accuracy_score(y, model.predict(x))

def score_roc_auc(model, x, y): 
    return metrics.roc_auc_score(y, model.predict_proba(x)[:,1])

def score_specificity(model, x, y): 
    return metrics.precision_score(y, model.predict(x))

def score_sensitivity(model, x, y): 
    return metrics.recall_score(y, model.predict(x))

SCORES = {
    "accuracy": score_accuracy, 
    "roc_auc": score_roc_auc, 
    "specificity": score_specificity,
    "sensitivity": score_sensitivity,
}

In [ ]:
def cv_generator(): 
    yield(list(range(len(train_set))), list(range(len(train_set), len(train_set) + len(validation_set))))

In [ ]:
best_acc = 0
best_model = None
for j, c in enumerate(CLASSIFIERS): 
    model_best = 0
    model_acc = 0
    for i in tqdm(range(1000)): 
        clf = c(random_state=i, **(PARAMETERS[j]))
        clf.fit(train_set, train_labels)
        score = SCORES[SCORE](clf, validation_set, validation_labels)
        if score > best_acc:
            best_acc = score
            best_model = clf
        if score > model_acc:
            model_acc = score
            model_best = clf
    print(model_acc)
    print(model_best)
    print(best_acc)
    print(best_model)

In [ ]:
print("train accuracy: {}".format(best_model.score(train_fix_set, train_fix_labels)))
print("validation accuracy: {}".format(best_model.score(validation_set, validation_labels)))
print("test accuracy: {}".format(best_model.score(test_set, test_labels)))

In [ ]:
best_model = CalibratedClassifierCV(best_model, cv="prefit")

In [ ]:
best_model.fit(validation_set, validation_labels)

In [ ]:
def evaluation(name, dataset, labels): 
    model = best_model
    predictions = model.predict(dataset)
    probabilities = model.predict_proba(dataset)
    pm.record("{}_labels".format(name), list(labels))
    pm.record("{}_probabilities".format(name), list(probabilities[:,1]))
    pm.record("{}_predictions".format(name), list(predictions))
    print("accuracy: {}".format(metrics.accuracy_score(labels, predictions)))
    fig = plot_confusion_matrix_ensemble(labels, predictions, ["benign", "malignant"])
    fig.savefig("figures/ensemble-{}-confusion-matrix.svg".format(name), bbox_inches = "tight")
    fig.show()
    print(pandas.DataFrame(calculate_confusion_matrix_stats(labels, probabilities[:,1])))
    fig = plot_roc_curve(labels, probabilities[:,1])
    fig.savefig("figures/ensemble-{}-roc-curve.svg".format(name), bbox_inches = "tight")
    fig.show()
    fig = plot_precision_recall(labels, probabilities[:,1])
    fig.savefig("figures/ensemble-{}-precisionrecall.svg".format(name), bbox_inches = "tight")
    fig.show()

# Training

In [ ]:
evaluation("train", train_fix_set, train_fix_labels)

# Validation

In [ ]:
evaluation("validation", validation_set, validation_labels)

# Test

In [ ]:
evaluation("test", test_set, test_labels)

In [ ]:
evaluation("test-1", test_1_set, test_1_labels)
evaluation("test-2", test_2_set, test_2_labels)

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')